# Project Title
### Data Engineering Capstone Project

#### Project Summary
 This POC has been set up in order to see the possible business value that can be extracted from insights on the patterns of tourist visiting the USA. 
 The goal is to asses:
 - Easiness and value of business insights extraction 
 - Fesability in therms of human and technical capital 
 - Issues encountered and possible remediation techniques

Being part fo an interdisciplinary agile team, you have been charged during the spring to create a simple data pipeline transformation.


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import os
import datetime
import pandas as pd
import pyspark.sql as pysql
import pyspark.sql.functions as F

from pyspark.sql import SparkSession
from pyspark.sql.types import *

### Step 1: Scope the Project and Gather Data

#### Scope 
As initial focus, the major driving forces of this POC will be studying simple implications between:
- __Tourist__: characteristics and idiosincrasies  
- __Weather__: during the stayed moment (not taking into consideration _second order thoughts_,i.e. which weather would I think it will make __when__ I decide to purchase the flight)
- __Cities__: characteristics

#### Data Gathered
 - __I94 Immigration Data__: This [data](https://travel.trade.gov/research/reports/i94/historical/2016.html) comes from the US National Tourism and Trade Office. 
 - __World Temperature Data__: This [data](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data) informs us about the surface temperature by city.
 - __U.S: Demographic Data__: This [data](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/) ifnroms us about US cities demographics.
 - __Airport Code Table__: This [data](https://datahub.io/core/airport-codes#data), gives us detailed information of the airport codes and their characteristics.

In [2]:
spark = SparkSession.builder.config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11").enableHiveSupport().getOrCreate()

##### Airport Codes

In [3]:
airport_codes = pd.read_csv('airport-codes_csv.csv')
airport_codes = airport_codes[airport_codes['iso_country']=='US']
airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [4]:
airport_schema = StructType([
    StructField("indent", StringType()),
    StructField("type", StringType()),
    StructField("name", StringType()),
    StructField("elevation_ft", FloatType()),
    StructField("continent", StringType()),
    StructField("iso_country", StringType()),
    StructField("iso_region", StringType()),
    StructField("municipality", StringType()),
    StructField("gps_code", StringType()),
    StructField("iata_code", StringType()),
    StructField("local_code", StringType()),
    StructField("coordinates", StringType())
])

airport_codes = spark.read.csv('airport-codes_csv.csv', schema=airport_schema, header=True, mode='DROPMALFORMED')
airport_codes.printSchema()

root
 |-- indent: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: float (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [5]:
print(airport_codes.limit(1).collect())

[Row(indent='00A', type='heliport', name='Total Rf Heliport', elevation_ft=11.0, continent='NA', iso_country='US', iso_region='US-PA', municipality='Bensalem', gps_code='00A', iata_code=None, local_code='00A', coordinates='-74.93360137939453, 40.07080078125')]


 ##### U.S. City Demographic

In [6]:
us_city  = pd.read_csv('us-cities-demographics.csv', sep=';')
us_city.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [7]:
city_schema = StructType([
    StructField("City", StringType()),
    StructField("State", StringType()),
    StructField("Median Age", FloatType()),
    StructField("Male Population", IntegerType()),
    StructField("Female Population", IntegerType()),
    StructField("Total", IntegerType()),
    StructField("Number of Veterans", IntegerType()),
    StructField("Foreign-born", IntegerType()),
    StructField("Average Household Size", FloatType()),
    StructField("State Code", StringType()),
    StructField("Race", StringType()),
    StructField("Count", IntegerType())
])

us_city = spark.read.csv('us-cities-demographics.csv',  sep=';', schema=city_schema, header=True, mode='DROPMALFORMED')
us_city.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: float (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: float (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



In [8]:
print(us_city.limit(1).collect())

[Row(City='Silver Spring', State='Maryland', Median Age=33.79999923706055, Male Population=40601, Female Population=41862, Total=82463, Number of Veterans=1562, Foreign-born=30908, Average Household Size=2.5999999046325684, State Code='MD', Race='Hispanic or Latino', Count=25924)]


 ##### World Temperature Data

In [9]:
temperature  = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [10]:
temperature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [11]:
temperature_schema = StructType([
    StructField("dt", TimestampType()),
    StructField("AverageTemperature", FloatType()),
    StructField("AverageTemperatureUncertainty", FloatType()),
    StructField("City", StringType()),
    StructField("Country", StringType()),
    StructField("Latitude", StringType()),
    StructField("Longitude", StringType()),
])

temperature = spark.read.csv('../../data2/GlobalLandTemperaturesByCity.csv', schema=temperature_schema, header=True, mode='DROPMALFORMED')
temperature.printSchema()

root
 |-- dt: timestamp (nullable = true)
 |-- AverageTemperature: float (nullable = true)
 |-- AverageTemperatureUncertainty: float (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [12]:
print(temperature.limit(1).collect())

[Row(dt=datetime.datetime(1743, 11, 1, 0, 0), AverageTemperature=6.067999839782715, AverageTemperatureUncertainty=1.7369999885559082, City='Århus', Country='Denmark', Latitude='57.05N', Longitude='10.33E')]


 ##### I94 Immigration Data

In [13]:
immigration = pd.read_csv('immigration_data_sample.csv')
immigration.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [14]:
immigration_schema = StructType([
    StructField("cicid", DoubleType()),
    StructField("i94yr", DoubleType()),
    StructField("i94mon", DoubleType()),
    StructField("i94cit", DoubleType()),
    StructField("i94res", DoubleType()),
    StructField("i94port", StringType()),
    StructField("arrdate", DoubleType()),
    StructField("i94mode", DoubleType()),
    StructField("i94addr", StringType()),
    StructField("depdate", DoubleType()),
    StructField("i94bir", DoubleType()),
    StructField("i94visa", DoubleType()),
    StructField("count", DoubleType()),
    StructField("dtadfile", DoubleType()),
    StructField("visapost", StringType()),
    StructField("occup", StringType()),
    StructField("entdepa", StringType()),
    StructField("entdepd", StringType()),
    StructField("entdepu", StringType()),
    StructField("matflag", StringType()),
    StructField("biryear", DoubleType()),
    StructField("dtaddto", StringType()),
    StructField("gender", StringType()),
    StructField("insnum", StringType()),
    StructField("airline", StringType()),
    StructField("admnum", DoubleType()),
    StructField("fltno", StringType()),
    StructField("visatype", StringType()),
])

immigration = None
immigration_columns = None
for file in os.listdir('../../data/18-83510-I94-Data-2016'):
        if not immigration:
            immigration = spark.read.load(path='../../data/18-83510-I94-Data-2016/' + file, format='com.github.saurfang.sas.spark', schema=immigration_schema, header=True, mode='DROPMALFORMED')
            immigration_columns = immigration.columns
        else:
            immigration_temp = spark.read.load(path='../../data/18-83510-I94-Data-2016/' + file, format='com.github.saurfang.sas.spark', schema=immigration_schema, header=True, mode='DROPMALFORMED')
            immigration_temp = immigration_temp.select(immigration_columns)
            immigration= immigration.unionByName(immigration_temp)
immigration.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: double (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [15]:
print(immigration.limit(1).collect())

[Row(cicid=6.0, i94yr=2016.0, i94mon=4.0, i94cit=692.0, i94res=692.0, i94port='XXX', arrdate=20573.0, i94mode=None, i94addr=None, depdate=None, i94bir=37.0, i94visa=2.0, count=1.0, dtadfile=None, visapost=None, occup=None, entdepa='T', entdepd=None, entdepu='U', matflag=None, biryear=1979.0, dtaddto='10282016', gender=None, insnum=None, airline=None, admnum=1897628485.0, fltno=None, visatype='B2')]


### Step 2: Explore and Assess the Data
#### Explore the Data 
Data exploration is to be hold at an absolute minimum. Due to the "continuous nature" of data pipelines, it is not performant to establish manual controls; the best approach is to establish automatic data cleaning systems.
This pipeline prepares the golden datasets and its derived tables, extensive data exploration is to be held once it is produced.


#### Cleaning Steps
##### Airport Codes
For the `airport codes` table, we are solely interested in the _bridge information_ it contains relating the __airport codes__ to the diferent __cities__ names. For this data just a simple string processing to remove any 
caps, to remove the likelihood of not having successful joins by `caps-missmatch

In [16]:
# Select required columns
airport_codes = airport_codes.select('iso_country', 'municipality', 'iata_code')

# Small string processing
airport_codes = airport_codes.withColumn('iso_country', F.lower(airport_codes.iso_country))\
                             .withColumn('municipality', F.lower(airport_codes.municipality))\
                             .withColumn('iata_code', F.lower(airport_codes.iata_code))

# NAs and duplicates processing
airport_codes = airport_codes.dropna(how='any')
airport_codes = airport_codes.dropDuplicates()

# Filtering of data to specific scope
airport_codes = airport_codes.filter(airport_codes.iso_country == 'US')

# Remove unnecessary columns
airport_codes = airport_codes.drop('iso_country')

# Renaming to decided conventions
airport_codes = airport_codes.withColumnRenamed('municipality', 'city')

 ##### U.S. City Demographic
The `U.S. City Demographic` contains much more useful information for the expected analysis, some columns are removed due to being redundant and other due to have non scientific base.

In [17]:
# Select required columns
us_city = us_city.select('City', 'Median Age', 'Male Population', 'Female Population', 'Total', 'Foreign-born', 'Average Household Size')

# Renaming to decided conventions
us_city = us_city.withColumnRenamed('City', 'city')\
                 .withColumnRenamed('Median Age', 'city_age')\
                 .withColumnRenamed('Male Population', 'population_male')\
                 .withColumnRenamed('Female Population', 'population_female')\
                 .withColumnRenamed('Total', 'population')\
                 .withColumnRenamed('Foreign-born', 'population_no_native')\
                 .withColumnRenamed('Average Household Size', 'house_size')

# NAs and duplicates processing
us_city = us_city.dropna(how='any')
us_city = us_city.dropDuplicates()

# Small string processing
us_city = us_city.withColumn('city', F.lower(us_city.city))

 ##### World Temperature
 For the world temperature we will select just the temperature information related data and the cities as key to union with other tables

In [18]:
# Select required columns
temperature = temperature.select('dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City', 'Country')

# Renaming to decided conventions
temperature = temperature.withColumnRenamed('dt', 'datetime')\
                 .withColumnRenamed('AverageTemperature', 'temperature')\
                 .withColumnRenamed('AverageTemperatureUncertainty', 'temperature_uncertainty')\
                 .withColumnRenamed('City', 'city')\
                 .withColumnRenamed('Country', 'country')

# Filtering of data to specific scope
temperature = temperature.filter(temperature.country == 'United States')

min_date = datetime.datetime(2016, 1, 1, 0, 0)
max_date = datetime.datetime(2017, 1, 1, 0, 0)
temperature = temperature.filter(temperature.datetime >= min_date)\
                         .filter(temperature.datetime < max_date)

# Remove unnecessary columns
temperature = temperature.drop('country')

# NAs and duplicates processing
temperature = temperature.dropna(how='any')
temperature = temperature.dropDuplicates()

# Small string processing
temperature = temperature.withColumn('city', F.lower(temperature.city))

##### I94 Immigration Data
The immigration data, contains many data relating the travelers and `border security`relevant features.
This dataset analysis of the different fields remains pretty evasive, as no source of the data fields is
provided; neither it is ´obvious'(to the developer) to understand their meaning

In [19]:
# Select required columns
immigration = immigration.select('i94yr', 'i94mon', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'biryear', 'gender', 'airline', 'visatype')

# Renaming to decided conventions
immigration = immigration.withColumnRenamed('i94yr', 'year')\
                         .withColumnRenamed('i94mon', 'month')\
                         .withColumnRenamed('i94port', 'iata_code')\
                         .withColumnRenamed('arrdate', 'arrival_date')\
                         .withColumnRenamed('i94mode', 'mode')\
                         .withColumnRenamed('i94addr', 'state_address')\
                         .withColumnRenamed('depdate', 'departure_date')\
                         .withColumnRenamed('i94bir', 'age')\
                         .withColumnRenamed('i94visa', 'visa')\
                         .withColumnRenamed('biryear', 'year_born')\
                         .withColumnRenamed('visatype', 'visa_type')

# Filtering of data to specific scope
immigration = immigration.filter(immigration.year == 2016)

# Remove unnecessary columns
immigration = immigration.drop('year')

# NAs and duplicates processing
immigration = immigration.dropna(how='any')
immigration = immigration.dropDuplicates()

# Small string processing
immigration = immigration.withColumn('iata_code', F.lower(immigration.iata_code))\
                         .withColumn('state_address', F.lower(immigration.state_address))\
                         .withColumn('gender', F.lower(immigration.gender))\
                         .withColumn('airline', F.lower(immigration.airline))\
                         .withColumn('visa_type', F.lower(immigration.visa_type))


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The choosen data model joins the different tables, their structure is:
- temperature: this table is a facts table
- immigration: this is a fact table
- us_city: this is a dimentional table
- airport_codes: this is a dimentional table

Having not a _unique_ facts table along, to create a star diagram structure;
we can still create something that resembles it by having as central part of 
our structure the `immigrations`table.

A code representation of the structure would be:

In [20]:
dataset = immigration.join(airport_codes, "iata_code")\
                     .join(temperature, "city")\
                     .join(us_city, "city")
dataset.columns

['city',
 'iata_code',
 'month',
 'arrival_date',
 'mode',
 'state_address',
 'departure_date',
 'age',
 'visa',
 'year_born',
 'gender',
 'airline',
 'visa_type',
 'datetime',
 'temperature',
 'temperature_uncertainty',
 'city_age',
 'population_male',
 'population_female',
 'population',
 'population_no_native',
 'house_size']

This model has been choosen as a __vanilla model__, it makes no assumptions on the kind of query/analysis
that will be processed, neither on the type of technical access specifications (in therms of users and/or
SLAs). The data is simply all available and all together.

#### 3.2 Mapping Out Data Pipelines
The proposed pipeline is synthesized in the following parts:
1. __Landing__: data loading on the spark sessions
2. __Staging__: data aggregation in a single Dataframe (in this case just for ´immigration´ data)
3. __Data Processing__: in this steps we will devise several sub-steps
    +  __Golden datasets__: golden datasets creation consisted on the step of data cleaning for each specific data source (still not merged together)
    + __Derived datasets__: in this case there is just one single derived dataset, with all information available (data model _pseudo-star_).

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Data model creation is performed in the above section __3.1__.


#### 4.2 Data Quality Checks
Data quality checks can be performed in two ways: 
- __Data Content__: checks related to ensure we are not introducing in a given field, values we can ensure (by the typology of the source) to be incorrect. This can be:
    + _Idependent data values_ based, meaning we know each specific value must enforce certain parameters (age > 0 ...) and it is independent of the others values
    + _Dependent data values_ based, on an attribute's values on its ensemble, e.g. big deviations of the mean value (above 1.5 standard deviation) for a specific attribute from one ingestion to the previous one
- __Metadata__: checks related with the data structure and integrity maintenance as it is moving through the pipeline, this checks include:
    + Row counts
    + Checksums
    + ...
- __Pipeline__: checks create to ensure the different data pipelines code and transformations really perform as expected:
    + Database integrity checks
    + Unit tests
    + Behaviour tests
    + ...


On the different kind of checks applicable to this `data pipeline`, the scope will be reduced to _data content_ ones.
The _Metadata_ type is not going to be considered as we are working on a single Spark instance, and it is to be considered
that on this environment this king of __data corruption__ is not likely to ocurre (check already developed into the framework).
As fo the _pipeline_ type, this is not a proper pipeline encompassing several systems on different machines, hence they are not
going to be studied; still the ones derived from __improper coding__ is being review by the developer while creating this 
Jupyter notebook.

##### Airport Codes
Has no room for verification, vis a vis of the data it contains:
- city
- iata code


##### U.S. City Demographic
Can have some data checks related to the `population` and `age` attributes:
- city
- city_age
- male population
- female population
- population
- non native population
- house size

In [21]:
# Age
us_city = us_city.filter(us_city.city_age >= 0)\
                 .filter(us_city.city_age <= 130)

# Population
us_city = us_city.withColumn('check_failed', us_city.population != (us_city.population_male + us_city.population_female))
us_city = us_city.withColumn('check_failed', (us_city.check_failed) | (us_city.population < us_city.population_no_native))

us_city = us_city.filter(us_city.check_failed != True)
us_city = us_city.drop('check_failed')

 ##### World Temperature
 This table can have data checks related to the temperature vaues; the ´datetime´ have already been enforced by the 
 conversion procedure:
 - datetime
 - temperature
 - temperature_uncertainty
 - city

In [22]:
# Temperature
temperature = temperature.filter(temperature.temperature > -60)\
                         .filter(temperature.temperature < 60)

##### I94 Immigration Data
Can have some checks related to the `dates` of arrival and departure and `age` of the arriving passengers:
- month
- iata_code
- arrival_date
- mode
- state_address
- departure_date
- age
- visa
- year_born
- gender
- airline
- visa_type

In [23]:
# Date
immigration = immigration.filter(immigration.month >= 1)\
                         .filter(immigration.month <= 12)\
                         .filter(immigration.arrival_date >= immigration.departure_date)

# Age
immigration = immigration.filter(immigration.age >= 0)\
                         .filter(immigration.age <= 130)\
                         .filter(immigration.year_born >= 1890)\
                         .filter(immigration.year_born <= 2020)

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

The data dictionary for `dataset` table data model, is:
- city: city name; _source `us_city` and `airport_codes`_
- iata_code: airport iata codes;  _source `airport_codes`_
- month: month of the year; _source `immigration`_
- arrival_date: visitor arrival date (format unknown and no information provided on it); _source `immigration`_
- mode: visa modality (format unknown and no information provided on it); _source `immigration`_
- state_address: US state from the address provided; _source `immigration`_ 
- departure_date: visitor departure date (format unknown and no information provided on it); _source `immigration`_
- age: visitors age; _source `immigration`_
- visa: visitor's visa;  _source `immigration`_
- year_born: visitor's year of birth; _source `immigration`_
- gender: visitor's gender; _source `immigration`_
- airline: visitor's airline; _source `immigration`_
- visa_type: visitor's visa type;  _source `immigration`_
- datetime: datetime of the temperature information;  _source `temperature`_
- temperature: global average land temperature in celsius;  _source `temperature`_
- temperature_uncertainty: the 95% confidence interval around the average;  _source `temperature`_
- city_age: city's median age; _source `us_city`_
- population_male: city's median male age; _source `us_city`_
- population_female: city's median female age; _source `us_city`_
- population: city's population; _source `us_city`_
- population_no_native: city's non-native population; _source `us_city`_
- house_size: city's median house size (unit of measure not specified, may be __rooms__); _source `us_city`_

In [24]:
immigration.columns

['month',
 'iata_code',
 'arrival_date',
 'mode',
 'state_address',
 'departure_date',
 'age',
 'visa',
 'year_born',
 'gender',
 'airline',
 'visa_type']

In [25]:
airport_codes.columns

['city', 'iata_code']

In [26]:
us_city.columns

['city',
 'city_age',
 'population_male',
 'population_female',
 'population',
 'population_no_native',
 'house_size']

In [27]:
temperature.columns

['datetime', 'temperature', 'temperature_uncertainty', 'city']

In [28]:
dataset.columns

['city',
 'iata_code',
 'month',
 'arrival_date',
 'mode',
 'state_address',
 'departure_date',
 'age',
 'visa',
 'year_born',
 'gender',
 'airline',
 'visa_type',
 'datetime',
 'temperature',
 'temperature_uncertainty',
 'city_age',
 'population_male',
 'population_female',
 'population',
 'population_no_native',
 'house_size']

#### Step 5: Complete Project Write Up
##### Tools and Technologies rational
This project is a `toy-project`. Having working experience on real data engineering projects, this involve lots of stake holders and departments, alas of many different technology stacks.
Hence the selection of doing it __solely__ on _PySpark_ was motivated due t the fact that replicating it like in _"in real life"_ would be too much of an overhead not so much related on
`data engineering` skills but more into the _infrastructure_ and _business analyst_ side. 


##### Data Pipeline Scheduling
Being a `toy-project`based on independent files load and no __permanent state__ databases, this means running it every time and saving the results of the tables
into a file or database system. In resume the scheduling is __on demand__.


##### Scenarios Approach
For all of the scenarios there is a base template that will need to be fine tuned for each case specific demands. Generally speaking, the structure would consist 
of:
- __Landing zone__: here files arrive and are stored temporary, _acts as entrance buffer. 
- __Staging zone__: here files have already passed several basic checks (data quality), in order to ensure that different data generation sources are passing data in the agreed format and data does not contain  unexpected values
- __Golden Datasets__: in this step information of files is introduced into a structured database
- __Derived Datasets__: in this step, users, which may be _power business analysts or data engineers/scientist_, take the golden tables and combine them in the required way for their specific business case/ project.

This is a pipeline based on a file ingestion approach, there can also be direct DB pulls and/or stream (Kafka) feed. During all pipeline steps data can be saved in accordance to the code used to generate the original/derived
datasets; to have a full data lineage with historical acces capabilities (this me be useful not only for project, also for compliance and audit purposes). The different data storage can additionally have several saving
patterns, i.e. moving data from more available storage (and expensive) to less available one, depending on: creation_date, last modified date, ... As a schedule to oversea all of this operations I would incline myself to 
use `Apache Airflow`. 
Lastly, though not less important in a business environment it is required to have proper `data governance` and a proper `data model` of the department or company established, along with a clear `data ontology`

###### Scenario 1: data explodes
_In case there was a data explosion by 100x._
For this case I will go to the cloud, AWS in this case is selected, as an option that allows us to scale quickly and be able to handle __peak loads__ without requiring us to pay for valley hours (and requiring by up front investments).
- __Landing zone__: will be a comgination of `airflow workers` and `S3` for the file storage
- __Staging zone__: will be again based on ``airflow workers` and `S3` for the file storage
- __Golden Datasets__: will be based on an `Redshift` architecture
- __Derived Datasets__: will be based on an `Redshift` architecture

This as the main pipeline, obviously depending on the desires of enhanced __data lineage__ capabilities, the usage more capabilities is envisioned:
- S3 glacier
- ...

###### Scenario 2: data dasboard
_The data populates a dashboard that must be updated on a daily basis by 7am every day._
In this case the main focus is on data being feed daily; taking the assumption of data not exploding.

- __Landing zone__: will be a comgination of `airflow workers` and `S3` for the file storage
- __Staging zone__: will be again based on ``airflow workers` and `S3` for the file storage
- __Golden Datasets__: will be based on an `Redshift` architecture
- __Derived Datasets__: will be based on an `Redshift` architecture

As an additional point, depending on the dashboard requirements, we may change the __derived datastes__ `Redshift database` for:
- __[Apache Impala](https://impala.apache.org/)__: Impala provides low latency and high concurrency for BI/analytic queries on Hadoop 
- __[Apache Kudu](https://kudu.apache.org/)__ : Kudu provides a combination of fast inserts/updates and efficient columnar scans to enable multiple real-time analytic workloads across a single storage layer. 
- __[Apache Kylin](https://kylin.apache.org/)__ for __OLAP__ cubes in big data environment. Kylin is able to achieve near constant query speed regardless of the ever-growing data volume. Reducing query latency from minutes to sub-second, Kylin brings online analytics back to big data.


###### Scenario 3: data concurrency
The database needed to be accessed by 100+ people.

In this case the __strongest recommendation__ is to see the type of workload that this users will perfomr to select the best between: database, configuration  and coding. 
In general distributed databases with no master node, it may cause _bottle necks on the master node_ if many petitions are done simultaneously. 

Selecting as initial recommendations:
- __[Apache HBase](https://hbase.apache.org/)__ : you need random, realtime read/write access to your Big Data. 
- __[Apache Cassandra](https://cassandra.apache.org/)__: scalability and high availability without compromising performance. Linear scalability and proven fault-tolerance on commodity hardware or cloud infrastructure make it the perfect platform for mission-critical data. 